## Отчёт о продажах по реализации

Method: https://statistics-api.wildberries.ru/api/v5/supplier/reportDetailByPeriod

Link: https://dev.wildberries.ru/openapi/financial-reports-and-accounting#tag/Finansovye-

Описание метода
Метод предоставляет детализации к еженедельным отчётам реализации.

Данные доступны с 29 января 2024 года.

Максимум 1 запрос в минуту на один аккаунт продавца

query Parameters
dateFrom
required
string <RFC3339>
Начальная дата отчёта.
Дата в формате RFC3339. Можно передать дату или дату со временем. Время можно указывать с точностью до секунд или миллисекунд.
Время передаётся в часовом поясе Мск (UTC+3).
Примеры:

2019-06-20
2019-06-20T23:59:59
2019-06-20T00:00:00.12345
2017-03-25T00:00:00
limit	
integer
Default: 100000
Максимальное количество строк ответа, возвращаемых методом. Не может быть более 100000.

dateTo
required
string <date>
Конечная дата отчёта

rrdid	
integer
Уникальный ID строки отчёта. Необходим для получения отчёта частями.
Загрузку отчёта нужно начинать с rrdid = 0 и при последующих вызовах API передавать в запросе значение rrd_id из последней строки, полученной в результате предыдущего вызова.
Таким образом, для загрузки одного отчёта может понадобиться вызывать API до тех пор, пока в ответе не будет отдан пустой массив [].

In [1]:
import requests
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
from clickhouse_connect import get_client
from clickhouse_driver import Client
import time
from typing import List, Dict

load_dotenv()

True

In [2]:
KeySmart = os.getenv("KeySmart")
password = os.getenv('ClickHouse')

In [3]:

class WildberriesAPI:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://statistics-api.wildberries.ru/api/v5/supplier/reportDetailByPeriod"
        self.last_request = 0
    
    def _wait_if_needed(self):
        """Wait if less than 60 seconds since last request"""
        elapsed = time.time() - self.last_request
        if elapsed < 60:
            time.sleep(60 - elapsed)
    
    def get_report(self, date_from: str, date_to: str, limit: int = 100000) -> List[Dict]:
        """Fetch report with automatic pagination"""
        results = []
        rrdid = 0
        
        while True:
            self._wait_if_needed()
            
            params = {
                "dateFrom": date_from,
                "dateTo": date_to,
                "limit": min(limit, 100000),
                "rrdid": rrdid
            }
            
            response = requests.get(
                self.base_url,
                headers={"Authorization": self.api_key},
                params=params
            )
            
            self.last_request = time.time()
            
            if not response.ok:
                print(f"Error: {response.status_code}")
                break
                
            data = response.json()
            if not data:
                break
                
            results.extend(data)
            rrdid = data[-1].get("rrd_id", 0)
            
            if len(data) < limit:
                break
        
        return results


# Example usage
if __name__ == "__main__":
    api = WildberriesAPI(api_key=KeySmart)
    
    report = api.get_report(
        date_from="2025-04-21",
        date_to="2025-05-05"
    )
    
    print(f"Got {len(report)} records")
    if report:
        print("First record:", report[0])
        
    # Convert to pandas DataFrame
    df = pd.DataFrame(report)

    # Basic data cleaning
    df = df.applymap(lambda x: None if x == '' else x)  # Replace empty strings with None# Add missing columns with default values
    
    if 'load_dt' not in df.columns:
        df['load_dt'] = pd.Timestamp.now()
    if 'source' not in df.columns:
        df['source'] = "WB-Realization-API"



Got 4492 records
First record: {'realizationreport_id': 346445542, 'date_from': '2025-04-21', 'date_to': '2025-04-27', 'create_dt': '2025-04-28', 'currency_name': 'KZT', 'suppliercontract_code': None, 'rrd_id': 2915427225786, 'gi_id': 27951818, 'dlv_prc': 1.2, 'fix_tariff_date_from': '2025-04-10T21:00:00Z', 'fix_tariff_date_to': '2025-06-09T21:00:00Z', 'subject_name': 'Триммеры', 'nm_id': 253299215, 'brand_name': 'Braun', 'sa_name': '4210201447870', 'ts_name': '0', 'barcode': '4210201447870', 'doc_type_name': '', 'quantity': 0, 'retail_price': 0, 'retail_amount': 0, 'sale_percent': 0, 'commission_percent': 0, 'office_name': 'Краснодар', 'supplier_oper_name': 'Логистика', 'order_dt': '2025-04-17T03:30:45Z', 'sale_dt': '2025-04-21T03:20:52Z', 'rr_dt': '2025-04-21', 'shk_id': 32460922593, 'retail_price_withdisc_rub': 0, 'delivery_amount': 1, 'return_amount': 0, 'delivery_rub': 380.96, 'gi_box_type_name': 'Микс', 'product_discount_for_report': 0, 'supplier_promo': 0, 'rid': 0, 'ppvz_spp_pr

C:\Users\User\AppData\Local\Temp\ipykernel_16296\3367259505.py:70: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if x == '' else x)  # Replace empty strings with None# Add missing columns with default values


In [4]:
# Export the DataFrame to an Excel file
output_file = "WB_Realization_API.xlsx"  # Specify the output file name
df.to_excel(output_file, index=False)  # Save without the index column

print(f"DataFrame successfully saved to {output_file}")
df

DataFrame successfully saved to WB_Realization_API.xlsx


,realizationreport_id,date_from,date_to,create_dt,currency_name,suppliercontract_code,rrd_id,gi_id,dlv_prc,fix_tariff_date_from,...,assembly_id,srid,report_type,is_legal_entity,trbx_id,installment_cofinancing_amount,wibes_wb_discount_percent,rebill_logistic_org,load_dt,source
0,346445542,2025-04-21,2025-04-27,2025-04-28,KZT,None,2915427225786,27951818,1.20,2025-04-10T21:00:00Z,...,0,27852088107407922.0.0,1,False,0,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
1,346445542,2025-04-21,2025-04-27,2025-04-28,KZT,None,2915427225787,27951818,0.00,None,...,0,27852088107407922.0.0,1,False,None,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
2,346445542,2025-04-21,2025-04-27,2025-04-28,KZT,None,2915427225788,26732473,2.20,2025-02-05T21:00:00Z,...,0,5024727614622832039.0.0,1,False,0,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
3,346445542,2025-04-21,2025-04-27,2025-04-28,KZT,None,2915427225789,26732473,0.00,None,...,0,5024727614622832039.0.0,1,False,None,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
4,346445542,2025-04-21,2025-04-27,2025-04-28,KZT,None,2915427225790,27428148,1.05,2025-03-03T21:00:00Z,...,0,8893189404925762064.2.0,1,False,0,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487,352760899,2025-04-28,2025-05-04,2025-05-05,KZT,None,2917624538971,0,0.00,None,...,0,7445702655142730102.0.0,1,False,None,0,0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МОДУ...",2025-05-05 10:40:48.690537,WB-Realization-API
4488,352760899,2025-04-28,2025-05-04,2025-05-05,KZT,None,2917624538972,0,0.00,None,...,0,9088590359107207918.0.0,1,False,None,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
4489,352760899,2025-04-28,2025-05-04,2025-05-05,KZT,None,2917624538973,0,0.00,None,...,0,15400288107976180.1.0,1,False,None,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API
4490,352760899,2025-04-28,2025-05-04,2025-05-05,KZT,None,2917624538974,0,0.00,None,...,0,dj.i41c6db197fe146aa8e7e940fc6d7a84c.0.0,1,False,None,0,0,NaN,2025-05-05 10:40:48.690537,WB-Realization-API


In [5]:
import logging
import numpy as np

df = df.copy()

# Ensure the DataFrame has the correct columns
columns = [
    'realizationreport_id', 'date_from', 'date_to', 'create_dt', 'currency_name',
    'suppliercontract_code', 'rrd_id', 'gi_id', 'dlv_prc', 'fix_tariff_date_from',
    'fix_tariff_date_to', 'subject_name', 'nm_id', 'brand_name', 'sa_name', 'ts_name',
    'barcode', 'doc_type_name', 'quantity', 'retail_price', 'retail_amount',
    'sale_percent', 'commission_percent', 'office_name', 'supplier_oper_name',
    'order_dt', 'sale_dt', 'rr_dt', 'shk_id', 'retail_price_withdisc_rub',
    'delivery_amount', 'return_amount', 'delivery_rub', 'gi_box_type_name',
    'product_discount_for_report', 'supplier_promo', 'rid', 'ppvz_spp_prc',
    'ppvz_kvw_prc_base', 'ppvz_kvw_prc', 'sup_rating_prc_up', 'is_kgvp_v2',
    'ppvz_sales_commission', 'ppvz_for_pay', 'ppvz_reward', 'acquiring_fee',
    'acquiring_percent', 'payment_processing', 'acquiring_bank', 'ppvz_vw',
    'ppvz_vw_nds', 'ppvz_office_name', 'ppvz_office_id', 'ppvz_supplier_id',
    'ppvz_supplier_name', 'ppvz_inn', 'declaration_number', 'bonus_type_name',
    'sticker_id', 'site_country', 'srv_dbs', 'penalty', 'additional_payment',
    'rebill_logistic_cost', 'storage_fee', 'deduction', 'acceptance', 'assembly_id',
    'srid', 'report_type', 'is_legal_entity', 'trbx_id', 'rebill_logistic_org',
    'load_dt', 'source'
]

# Convert date columns to datetime and handle NaT values
date_cols = ['date_from', 'date_to', 'create_dt', 'fix_tariff_date_from',
             'fix_tariff_date_to', 'order_dt', 'sale_dt', 'rr_dt', 'load_dt']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.tz_localize(None)
        df[col] = df[col].where(pd.notnull(df[col]), None)  # Replace NaT with None

# Convert boolean columns to integers
df['srv_dbs'] = df['srv_dbs'].astype(int)
df['is_legal_entity'] = df['is_legal_entity'].astype(int)

# Handle nullable string columns
nullable_cols = [
    'suppliercontract_code', 'fix_tariff_date_from', 'fix_tariff_date_to',
    'subject_name', 'brand_name', 'sa_name', 'ts_name', 'barcode', 'doc_type_name',
    'office_name', 'gi_box_type_name', 'payment_processing', 'acquiring_bank',
    'ppvz_office_name', 'ppvz_supplier_name', 'ppvz_inn', 'declaration_number',
    'bonus_type_name', 'site_country', 'srid', 'trbx_id', 'rebill_logistic_org'
]

for col in nullable_cols:
    df[col] = df[col].replace({np.nan: None, '': None})

# Reorder columns to match the expected order
data_organized = df[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in data_organized.to_numpy()]

In [8]:
from datetime import  date
today = date.today()
previous_monday = today - timedelta(days=today.weekday() + 7)
# Calculate the Sunday of the previous week
previous_sunday = previous_monday + timedelta(days=6)
print(previous_monday, " - ", previous_sunday)

2025-04-28  -  2025-05-04


In [7]:
# Define the table name
table_name = 'wb_realization_reports'

# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                   # Your ClickHouse username
    password=password,                                  # Your ClickHouse password
    database='user1',                                   # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification
)

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
logging.info("Data inserted successfully!")